# Welcome to the BQ SQL Performance Challenge!
> Authors:
> - Ashish Modi (asmodi@google.com)
> - Eri Santos (erisantos@google.com)
> - Ruben Fernandez (fernandezruben@google.com)

> Chopping board:
> - Vadim Zaripov (vadz@google.com)




### Instructions
**IMPORTANT:**

* **This lab must be run in Argolis logged in as `admin@<yourldap>.altostrat.com`**
* Before running this lab, please [**follow the click-to-deploy of this demo**](https://cloud-demo-hub.corp.google.com/demo/Argolis-Data-Share/618).
  * Deploying it will add the user `admin@<yourldap>.altostrat.com` to a group that has access to the dataset we'll be using.
* Run the cells under this **Setup** section to prepare the project for the lab work.
* This lab will create BQ reservations and assignments. Remember to run the *Cleanup* cells after you're done to remove them.

#### Why Argolis? - About the data
The lab uses a dataset shared only to the `admin@<yourldap>.altostrat.com` user (via BQ Sharing) after deploying the [go/demos demo mentioned above]((https://cloud-demo-hub.corp.google.com/demo/Argolis-Data-Share/618)). You can find the [BigQuery Sharing listing here](https://console.cloud.google.com/bigquery/analytics-hub/exchanges/projects/720965328418/locations/us/dataExchanges/argolis_shared_data_1840b70ffcb/listings/bigquery_optimization_lab_199a6715d4a).

 If you want to use different user, you can first link the dataset as `admin@`, and then share accordingly. Outside of the BQ Sharing linking steps, the rest of the notebook can run as the user of your choice.

## Setup

In [ ]:
# Variables for the lab

YOUR_PROJECT_ID = "YOUR_PROJECT_ID" # @param {type:"string"} # << == INPUT your bq project name for this lab
LINKED_DATASET_ID = "linked_sme_da"  # @param {type:"string"} # << == INPUT name you want for your linked dataset
YOUR_WORKING_DATASET = "sme_da_lab_working_dataset" # @param {type:"string"} # << == INPUT name you want for your local working dataset

In [ ]:
import sys
import time
from google.cloud import bigquery
from google.cloud.exceptions import NotFound
import google.auth
import google.auth.transport.requests
from google.api_core import exceptions
import humanize
import requests
import json

In [ ]:
# Do not modify
PROJECT_ID = YOUR_PROJECT_ID

# Do not modify
REGION = "US"

RESERVATION_NAME = "sme-academy-reservation"
reservation_url = f"projects/{PROJECT_ID}/locations/{REGION}/reservations/{RESERVATION_NAME}"

LISTING_ENDPOINT = "https://analyticshub.googleapis.com/v1/projects/720965328418/locations/us/dataExchanges/argolis_shared_data_1840b70ffcb/listings/bigquery_optimization_lab_199a6715d4a:subscribe"


### Subscribing to the dataset via BigQuery Sharing

In [ ]:
def subscribe_to_listing(listing_endpoint, project_id, linked_dataset_id, location):
    """
    Subscribes to a BigQuery Analytics Hub listing via the API.

    Args:
        listing_endpoint (str): The API endpoint for the listing subscription.
        project_id (str): The ID of the project where the destination dataset will be created.
        linked_dataset_id (str): The ID of the destination dataset.
        location (str): The location of the destination dataset.

    Returns:
        requests.Response: The response object from the API call.
    """
    credentials, project = google.auth.default()
    authed_session = google.auth.transport.requests.AuthorizedSession(credentials)

    payload = {
        "destinationDataset": {
            "datasetReference": {
                "projectId": project_id,
                "datasetId": linked_dataset_id
            },
            "location": location
        }
    }
    headers = {'Content-Type': 'application/json'}
    response = authed_session.post(listing_endpoint, data=json.dumps(payload), headers=headers)

    if response.status_code in [200, 201]:
      response_json = response.json()
      linked_dataset_name = response_json.get('name', 'N/A')
      print(f"Successfully subscribed to listing! Linked dataset name: {project_id}.{linked_dataset_id}")
    elif response.status_code == 409:
        print(f"Error 409: Conflict. The dataset '{PROJECT_ID}.{LINKED_DATASET_ID}' might already be linked or exist.")
        print("Please check if the dataset already exists or if there's an existing subscription.")
        print(f"Full error response: {response.json()}")
    else:
        print(f"An unexpected error occurred. Status Code: {response.status_code}")
        print(f"Full error response: {response.json()}")

    return response


# Create a linked dataset to the
response = subscribe_to_listing(LISTING_ENDPOINT, YOUR_PROJECT_ID, LINKED_DATASET_ID, REGION)

### Helper Functions

In [ ]:
def compare_bq_jobs(
    job_id_1: str,
    job_id_2: str,
    project_id: str = PROJECT_ID,
    location: str = REGION,
):
    """
    Retrieves and compares the statistics for two completed BigQuery jobs.

    Args:
     job_id_1: The job ID (e.g., 'bquxjob_...') for the first job.
     job_id_2: The job ID for the second job.
     project_id: (optional) The GCP project ID where the jobs were run,
         defaults to the PROJECT_ID variable.
     location: (optional) The location/region of the jobs (e.g., 'US', 'EU'),
         defaults to the REGION variable.
    """

    try:
        # Initialize the client
        # By default, this uses Application Default Credentials
        client = bigquery.Client(project=project_id)

        # Get job information
        job_1 = client.get_job(job_id_1, project=project_id, location=location)
        job_2 = client.get_job(job_id_2, project=project_id, location=location)

        # Validate that they are query jobs
        if not isinstance(job_1, bigquery.QueryJob) or not isinstance(
            job_2, bigquery.QueryJob
        ):
            print(
                "Error: One or both job IDs do not correspond to a QueryJob.",
                file=sys.stderr,
            )
            return

        # Validate that they are complete
        if job_1.state != "DONE" or job_2.state != "DONE":
            print(
                "Error: One or both jobs are not in the 'DONE' state.", file=sys.stderr
            )
            return

        if job_1.error_result or job_2.error_result:
            print("Warning: One or both jobs completed with errors.", file=sys.stderr)

        # Extract statistics
        stats_1 = {
            "id": job_1.job_id,
            "rows_returned": job_1.result().total_rows,
            "query_time": (
                (job_1.ended - job_1.started).total_seconds()
                if job_1.ended and job_1.started
                else 0
            ),
            "bytes_scanned": (
                job_1.total_bytes_processed
                if job_1.total_bytes_processed is not None
                else 0
            ),
            "slot_ms": job_1.slot_millis if job_1.slot_millis is not None else 0,
        }

        stats_2 = {
            "id": job_2.job_id,
            "rows_returned": job_2.result().total_rows,
            "query_time": (
                (job_2.ended - job_2.started).total_seconds()
                if job_2.ended and job_2.started
                else 0
            ),
            "bytes_scanned": (
                job_2.total_bytes_processed
                if job_2.total_bytes_processed is not None
                else 0
            ),
            "slot_ms": job_2.slot_millis if job_2.slot_millis is not None else 0,
        }

        # Print the comparison table
        print_comparison_table(stats_1, stats_2)

    except NotFound:
        print(
            "Error: Job not found. Check the job IDs, project, and location.",
            file=sys.stderr,
        )
    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        print(
            "Please ensure you are authenticated ('gcloud auth application-default login') "
            "and the job/project/location details are correct.",
            file=sys.stderr,
        )


# ----------------------------------------------------------------------------------------------- #


def compare_bq_jobs_static(
    job_1_dict: dict,
    job_id_2: str,
    project_id: str = PROJECT_ID,
    location: str = REGION,
):
    """
    Retrieves and compares the statistics for two completed BigQuery jobs.
    The first argument is not a job_id to be retrieved from BQ, but instead,
    it's a static dictionary with the job details.

    Use this method when you want to compare a job against a static baseline.

    Args:
     job_1: A dictionary with the job 1 execution details.
     job_id_2: The job ID for the second job.
     project_id: (optional) The GCP project ID where the jobs were run,
         defaults to the PROJECT_ID variable.
     location: (optional) The location/region of the jobs (e.g., 'US', 'EU'),
         defaults to the REGION variable.
    """

    try:
        # Initialize the client
        # By default, this uses Application Default Credentials
        client = bigquery.Client(project=project_id)

        # Get job information
        job_2 = client.get_job(job_id_2, project=project_id, location=location)

        # Validate that they are query jobs
        if not isinstance(
            job_2, bigquery.QueryJob
        ):
            print(
                "Error: One or both job IDs do not correspond to a QueryJob.",
                file=sys.stderr,
            )
            return

        # Validate that they are complete
        if job_2.state != "DONE":
            print(
                "Error: One or both jobs are not in the 'DONE' state.", file=sys.stderr
            )
            return

        if job_2.error_result:
            print("Warning: One or both jobs completed with errors.", file=sys.stderr)

        # Extract statistics
        stats_2 = {
            "id": job_2.job_id,
            "rows_returned": job_2.result().total_rows,
            "query_time": (
                (job_2.ended - job_2.started).total_seconds()
                if job_2.ended and job_2.started
                else 0
            ),
            "bytes_scanned": (
                job_2.total_bytes_processed
                if job_2.total_bytes_processed is not None
                else 0
            ),
            "slot_ms": job_2.slot_millis if job_2.slot_millis is not None else 0,
        }

        # Print the comparison table
        print_comparison_table(job_1_dict, stats_2)

    except NotFound:
        print(
            "Error: Job not found. Check the job IDs, project, and location.",
            file=sys.stderr,
        )
    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        print(
            "Please ensure you are authenticated ('gcloud auth application-default login') "
            "and the job/project/location details are correct.",
            file=sys.stderr,
        )


# ----------------------------------------------------------------------------------------------- #


def print_comparison_table(stats_1: dict, stats_2: dict):
    """Helper function to format and print an ASCII table comparing two BQ jobs."""

    # --- Helper functions for formatting ---
    def format_bytes(b: int) -> str:
        return humanize.naturalsize(b, binary=True)  # e.g., 1.2 GiB

    def format_time(s: float) -> str:
        return f"{s:,.2f} s"

    def format_slot_ms(ms: int) -> str:
        return f"{ms:,.0f} ms"

    def format_rows(r: int) -> str:
        if r is None:
            return "N/A"
        return f"{r:,.0f}"

    def get_comparison(val_1, val_2) -> str:
        """Calculates the percentage change between val_1 and val_2."""
        if val_1 is None and val_2 is None:
            return "N/A"
        if val_1 is None:  # val_2 must be not None
            return "(+inf %)" if val_2 > 0 else "(No change)"
        if val_2 is None:  # val_1 must be not None
            return "(-100.0%)" if val_1 > 0 else "(No change)"

        # From here, val_1 and val_2 are both not None (are numbers)
        if val_1 == 0:
            if val_2 > 0:
                return "(+inf %)"
            if val_2 < 0:
                return "(-inf %)"  # Should not happen for rows/bytes/time
            return "(No change)"  # val_1 == 0 and val_2 == 0

        percent = (val_2 - val_1) / val_1

        if percent == 0:
            return "(No change)"

        # For time, bytes, and slot_ms, lower is generally better
        comparison_text = "slower/more" if percent > 0 else "faster/less"
        return f"({percent:+.1%} {comparison_text})"

    # --- Prepare data for table ---
    data = [
        (
            "Query Time",
            stats_1["query_time"],
            stats_2["query_time"],
            format_time,
            get_comparison,
        ),
        (
            "Bytes Scanned",
            stats_1["bytes_scanned"],
            stats_2["bytes_scanned"],
            format_bytes,
            get_comparison,
        ),
        (
            "Slot Milliseconds",
            stats_1["slot_ms"],
            stats_2["slot_ms"],
            format_slot_ms,
            get_comparison,
        ),
        (
            "Rows Returned",
            stats_1["rows_returned"],
            stats_2["rows_returned"],
            format_rows,
            get_comparison,
        ),
    ]

    # --- Calculate column widths ---
    header_1 = f"Job 1 ({stats_1['id'][-12:]})"  # Show last 12 chars of ID
    header_2 = f"Job 2 ({stats_2['id'][-12:]})"

    formatted_job_1 = [row[3](row[1]) for row in data]
    formatted_job_2 = [row[3](row[2]) for row in data]
    formatted_comp = [row[4](row[1], row[2]) for row in data]

    col_1_width = max(len(row[0]) for row in data) + 2  # Metric
    col_2_width = max(max(len(s) for s in formatted_job_1), len(header_1)) + 2
    col_3_width = max(max(len(s) for s in formatted_job_2), len(header_2)) + 2
    col_4_width = max(max(len(s) for s in formatted_comp), len("Comparison")) + 2

    # --- Print Table ---
    # The total width for the title row needs to account for the
    # 3 extra '|' characters that separate the columns.
    total_width = col_1_width + col_2_width + col_3_width + col_4_width + 3

    def print_row(c1, c2, c3, c4, align_c1="left", pad_char=" "):
        c1_padded = (
            c1.ljust(col_1_width, pad_char)
            if align_c1 == "left"
            else c1.rjust(col_1_width, pad_char)
        )
        c2_padded = c2.rjust(col_2_width, pad_char)  # Right-align numbers
        c3_padded = c3.rjust(col_3_width, pad_char)
        c4_padded = c4.rjust(col_4_width, pad_char)
        print(f"|{c1_padded}|{c2_padded}|{c3_padded}|{c4_padded}|")

    def print_divider(char="="):
        print(
            f"+{char * col_1_width}+{char * col_2_width}+{char * col_3_width}+{char * col_4_width}+"
        )

    print("\n" + "+" + "=" * total_width + "+")
    title = "BigQuery Job Comparison"
    print(f"|{title:^{total_width}}|")
    print_divider()

    # Headers
    print_row(f" Metric ", f" {header_1} ", f" {header_2} ", f" Comparison ")
    print_divider("-")

    # Data Rows
    for i, row in enumerate(data):
        metric_name = f" {row[0]} "
        val_1_str = f" {formatted_job_1[i]} "
        val_2_str = f" {formatted_job_2[i]} "
        comp_str = f" {formatted_comp[i]} "
        print_row(metric_name, val_1_str, val_2_str, comp_str)

    print_divider("-")
    print()  # Newline at the end


# ----------------------------------------------------------------------------------------------- #


def get_job_details_infoschema(
    job_id: str = None, client: bigquery.Client = None
) -> None:
    """
    Fetches details of a BigQuery job from INFORMATION_SCHEMA.JOBS and prints
    it as a dataframe.

    Args:
     job_id: The job ID (e.g., 'bquxjob_...')
     client: (optional) The bigquery.Client object to run the INFORMATION_SCHEMA query
    """
    if client == None:
        client = bigquery.Client(project=PROJECT_ID)

    qs_table = f"region-{REGION}.INFORMATION_SCHEMA.JOBS"
    sql = f"SELECT * FROM `{qs_table}`"

    if job_id:
        sql += f" WHERE job_id = '{job_id}'"
    sql += f" AND project_id = '{PROJECT_ID}'"

    print(sql)

    df_result = client.query(sql).to_dataframe()
    print(df_result)


# ----------------------------------------------------------------------------------------------- #


def run_bq_query(
    query_sql: str,
    cache: bool = False,
    show_query_results: bool = False,
    project_id: str = PROJECT_ID,
    location: str = REGION,
) -> tuple [str | None, dict | None]:
    """
    Runs a BigQuery query, polls for completion, and prints job stats.

    Args:
     query_sql: The SQL query string to execute.
     cache: (optional) Whether to use BigQuery caching. Default: False
     show_query_results: If True, prints the first 10 rows of the result.
     project_id: (optional) The GCP project ID where the jobs were run, defaults to the PROJECT_ID variable.
     location: (optional) The location/region of the jobs (e.g., 'US', 'EU'), defaults to the REGION variable.

    Returns:
     A tuple containing:
     - The job ID as a string if the job started, otherwise None.
     - A dictionary containing job metadata if the job completed successfully, otherwise None.
    """
    print(f"Running query in project '{project_id}', location '{location}'...")
    try:
        client = bigquery.Client(project=project_id)

        # Start the query job
        job_config = bigquery.QueryJobConfig(
            priority=bigquery.QueryPriority.INTERACTIVE, use_query_cache=cache
        )

        job = client.query(query_sql, location=location, job_config=job_config)
        print(f"Job started. ID: {job.job_id}")

        job_start_time = time.time()
        # Wait for the job to complete, polling every 5 seconds
        while job.state in ("RUNNING", "PENDING"):
            elapsed = time.time() - job_start_time
            # \r moves the cursor to the start of the line for an overwriting update
            print(
                f" Job status: {job.state.lower()}... (elapsed: {elapsed:.0f}s) ",
                end="\r",
            )
            time.sleep(5)
            job.reload()

        # Clear the "running" line
        print(" " * 60, end="\r")

        # Print the final job summary
        print(f"Job {job.job_id} finished with state: {job.state}")

        if job.error_result:
            print("Error Details:", file=sys.stderr)
            print(job.error_result, file=sys.stderr)
            return job.job_id, None

        # Display job statistics if successful
        if job.state == "DONE":
            duration = (job.ended - job.started).total_seconds()
            bytes_processed_val = (
                job.total_bytes_processed
                if job.total_bytes_processed is not None
                else 0
            )
            bytes_scanned_formatted = humanize.naturalsize(
                bytes_processed_val, binary=True
            )
            slot_ms_val = job.slot_millis if job.slot_millis is not None else 0

            print("-" * 30)

            job_result = None # Initialize
            rows_returned = None

            if job.num_dml_affected_rows is not None:
                print(f" Rows Affected (DML): {job.num_dml_affected_rows:,}")
            else:
                # For SELECT queries, get total_rows from the result.
                # This will be 0 for DDL statements.
                try:
                    # Calling .result() waits for the job to complete and gets the RowIterator
                    job_result = job.result() # Get the result iterator
                    rows_returned = job_result.total_rows
                    print(f" Rows Returned:    {rows_returned:,}")
                except Exception as e:
                    # Handle cases like DDL where result() might not have total_rows
                    print(f" Rows Returned:    N/A (Not a SELECT query?)")


            # creating result dictionary
            job_metadata_result = {'job_id': job.job_id, 'duration': duration, 'bytes_scanned': bytes_scanned_formatted, 'slot_time': slot_ms_val, 'cache_hit': job.cache_hit}


            print(f" Duration:      {duration:,.2f} s")
            print(
                f" Bytes Scanned:    {bytes_scanned_formatted} ({bytes_processed_val:,} bytes)"
            )
            print(f" Slot Time:      {slot_ms_val:,.0f} ms")
            print("-" * 30)
            print()  # newline

            # Show query results
            if show_query_results:
                if job_result and rows_returned is not None and rows_returned > 0:
                    print("Query Results (first 10 rows):")
                    print("-" * 30)

                    try:
                        # Requires `pip install pandas db-dtypes`
                        dataframe = job_result.to_dataframe()
                        print(dataframe.head(10))
                        print("-" * 30)

                    except Exception as e:
                        print(f"Error printing query results with pandas: {e}", file=sys.stderr)
                        print("Please ensure 'pandas' and 'db-dtypes' libraries are installed.", file=sys.stderr)


        return job.job_id, job_metadata_result

    except NotFound:
        print(f"Error: Project or location may be incorrect.", file=sys.stderr)
        return None, None
    except Exception as e:
        print(f"An error occurred: {e}", file=sys.stderr)
        print(
            "Please ensure you are authenticated ('gcloud auth application-default login') "
            "and the project/location details are correct.",
            file=sys.stderr,
        )
        return None, None

In [ ]:
# Testing the compare_bq_jobs function
sql = """select 1"""

print("--- Running Job 1 ---")
job1_result = run_bq_query(sql)
job_id_1 = job1_result[0] if job1_result else None

print("\n--- Running Job 2 ---")
job2_result = run_bq_query(sql)
job_id_2 = job2_result[0] if job2_result else None

print("\n--- Comparing Jobs ---")
if job_id_1 and job_id_2:
    compare_bq_jobs(job_id_1, job_id_2)
else:
    print("Comparison skipped: One or both BQ jobs failed to return a valid job ID.")

### Additional Useful Information

If you want to see the ER Model diagram for these tables, clik [here](https://datacadamia.com/data/type/relation/benchmark/tpcds/schema).

Please find the exact DDL for the tables involved in this query below.


```
CREATE TABLE `<PROJECT>.<DATASET>.store_sales`
(
  ss_sold_date_sk INT64,
  ss_sold_time_sk INT64,
  ss_item_sk INT64,
  ss_customer_sk INT64,
  ss_cdemo_sk INT64,
  ss_hdemo_sk INT64,
  ss_addr_sk INT64,
  ss_store_sk INT64,
  ss_promo_sk INT64,
  ss_ticket_number INT64,
  ss_quantity INT64,
  ss_wholesale_cost FLOAT64,
  ss_list_price FLOAT64,
  ss_sales_price FLOAT64,
  ss_ext_discount_amt FLOAT64,
  ss_ext_sales_price FLOAT64,
  ss_ext_wholesale_cost FLOAT64,
  ss_ext_list_price FLOAT64,
  ss_ext_tax FLOAT64,
  ss_coupon_amt FLOAT64,
  ss_net_paid FLOAT64,
  ss_net_paid_inc_tax FLOAT64,
  ss_net_profit FLOAT64
);
```



```
CREATE TABLE `<PROJECT>.<DATASET>.customer_address`
(
  ca_address_sk INT64,
  ca_address_id STRING,
  ca_street_number STRING,
  ca_street_name STRING,
  ca_street_type STRING,
  ca_suite_number STRING,
  ca_city STRING,
  ca_county STRING,
  ca_state STRING,
  ca_zip STRING,
  ca_country STRING,
  ca_gmt_offset FLOAT64,
  ca_location_type STRING
);
```

```
CREATE TABLE `<PROJECT>.<DATASET>.household_demographics`
(
  hd_demo_sk INT64,
  hd_income_band_sk INT64,
  hd_buy_potential STRING,
  hd_dep_count INT64,
  hd_vehicle_count INT64
);
```

```
CREATE TABLE `<PROJECT>.<DATASET>.customer`
(
  c_customer_sk INT64,
  c_customer_id STRING,
  c_current_cdemo_sk INT64,
  c_current_hdemo_sk INT64,
  c_current_addr_sk INT64,
  c_first_shipto_date_sk INT64,
  c_first_sales_date_sk INT64,
  c_salutation STRING,
  c_first_name STRING,
  c_last_name STRING,
  c_preferred_cust_flag STRING,
  c_birth_day INT64,
  c_birth_month INT64,
  c_birth_year INT64,
  c_birth_country STRING,
  c_login STRING,
  c_email_address STRING,
  c_last_review_date_sk INT64
);
```

```
CREATE TABLE `<PROJECT>.<DATASET>.date_dim`
(
  d_date_sk INT64,
  d_date_id STRING,
  d_date DATE,
  d_month_seq INT64,
  d_week_seq INT64,
  d_quarter_seq INT64,
  d_year INT64,
  d_dow INT64,
  d_moy INT64,
  d_dom INT64,
  d_qoy INT64,
  d_fy_year INT64,
  d_fy_quarter_seq INT64,
  d_fy_week_seq INT64,
  d_day_name STRING,
  d_quarter_name STRING,
  d_holiday STRING,
  d_weekend STRING,
  d_following_holiday STRING,
  d_first_dom INT64,
  d_last_dom INT64,
  d_same_day_ly INT64,
  d_same_day_lq INT64,
  d_current_day STRING,
  d_current_week STRING,
  d_current_month STRING,
  d_current_quarter STRING,
  d_current_year STRING
);
```

```
CREATE TABLE `<PROJECT>.<DATASET>.store`
(
  s_store_sk INT64,
  s_store_id STRING,
  s_rec_start_date DATE,
  s_rec_end_date DATE,
  s_closed_date_sk INT64,
  s_store_name STRING,
  s_number_employees INT64,
  s_floor_space INT64,
  s_hours STRING,
  s_manager STRING,
  s_market_id INT64,
  s_geography_class STRING,
  s_market_desc STRING,
  s_market_manager STRING,
  s_division_id INT64,
  s_division_name STRING,
  s_company_id INT64,
  s_company_name STRING,
  s_street_number STRING,
  s_street_name STRING,
  s_street_type STRING,
  s_suite_number STRING,
  s_city STRING,
  s_county STRING,
  s_state STRING,
  s_zip STRING,
  s_country STRING,
  s_gmt_offset FLOAT64,
  s_tax_percentage FLOAT64
);
```

# BigQuery Optimization Techniques (15 mins)
### BQ SME Academy 2025
---
### 🎯 Your Goal

For each exercise, your goal is to run the queries and analyze the **"BigQuery Job Comparison"** table in the output. Pay close attention to these three key metrics:

* **Bytes Scanned:** The amount of data read from disk. This is the primary driver of query **cost**.
* **Slot Milliseconds:** The total compute time consumed by the query. This is a measure of query **complexity and speed**.
* **Query Time:** The wall-clock time it took for the query to complete.

The objective is to understand **why** these metrics change between the un-optimized and optimized queries.

### 💿 The Dataset

We will be using a 10TB version of the TPC-DS benchmark dataset

### 📝 How to use this section

1.  **Follow the Exercises:** Proceed through the numbered exercises in order.
2.  **Read the Explanation:** Before each code cell, read the markdown explanation of the anti-pattern and the proposed fix.
3.  **Execute the Queries:** Run the code cell for each exercise to see the performance comparison.
4.  **Analyze the Results:** Review the output table and make sure you understand why the metrics changed.

In [ ]:
# --- Setup -----
LAB01_DATASET_ID = LINKED_DATASET_ID


### (Optional) Enabling the Advanced Query Runtime

BigQuery's **advanced query runtime** is a project-level feature. It is designed to accelerate complex analytical queries, especially those with large joins, aggregations, and window functions, by using more dynamic resource management during execution.

For this lab, you can enable it to see if it provides an additional performance boost to the complex queries. This is a one-time setting for your project in a specific region.

#### How to Enable

You can enable the advanced runtime by executing the following SQL command.

**Note:** You must run this command in the specific region where you want the setting to apply

```python
# --- Enable Advanced Query Runtime ---

sql_enable_advanced_runtime = f"""
ALTER PROJECT `{PROJECT_ID}`
SET OPTIONS (
  `region-{REGION}.query_runtime` = "advanced"
);
"""

# To enable, uncomment and run the line below
# run_bq_query(sql_enable_advanced_runtime, location=LOCATION)

print("Advanced runtime enabled. You can now proceed with the lab exercises.")

```

#### How to Verify the Setting

You can check the current runtime setting for your project by running the following query in the appropriate region:
```sql
SELECT *
FROM `region-{REGION}`.INFORMATION_SCHEMA.PROJECT_OPTIONS
WHERE option_name = 'query_runtime';
```

In [ ]:
# --- Enable Advanced Query Runtime ---

sql_enable_advanced_runtime = f"""
ALTER PROJECT `{YOUR_PROJECT_ID}`
SET OPTIONS (
  `region-US.query_runtime` = "advanced"
);
"""

run_bq_query(sql_enable_advanced_runtime, location=REGION)

print("Advanced runtime enabled. You can now proceed with the lab exercises.")



In [ ]:
# --- Disable Advanced Query Runtime ---

sql_disable_advanced_runtime = f"""
ALTER PROJECT `{YOUR_PROJECT_ID}`
SET OPTIONS (
  `region-US.query_runtime` = NULL
);
"""

# To disable, uncomment and run the line below
# run_bq_query(sql_disable_advanced_runtime, location=REGION)

#print("Command to disable advanced runtime is ready to be executed.")

In [ ]:
sql_verify_runtime = f"""
SELECT option_name, option_value
FROM `region-US`.INFORMATION_SCHEMA.PROJECT_OPTIONS
WHERE option_name = 'query_runtime';
"""

run_bq_query(sql_verify_runtime, location=REGION, show_query_results=True)

### 1.  Selecting Unnecessary Columns (`SELECT *`)

**The Anti-Pattern:** Using `SELECT *` is one of the most common and costly mistakes in BigQuery. Because BigQuery is a columnar database, it charges based on the amount of data read from the columns you query. `SELECT *` forces BigQuery to read all data from every single column in the table, even if you only need a few.

**The Fix:** The solution is simple but highly effective: only select the specific columns you need for your analysis.

**Expected Outcome:** By selecting only the necessary columns, you dramatically reduce the amount of data processed. You should see a massive reduction in **Bytes Scanned**, which directly translates to lower query costs.

In [ ]:
# --- 1. SELECT * vs. SELECT specific_columns ---

# Note: We add a filter on a non-clustered field and a LIMIT to make the query runnable.
# The key is to observe the difference in "Bytes Scanned".

a1_unopt_query = f"""
-- Un-Optimized: Scans all 23 columns from the table.
SELECT
  *
FROM
  `{LAB01_DATASET_ID}.store_sales`
WHERE
  ss_ticket_number = 46153516
LIMIT 10;
"""

a1_opt_query = f"""
-- Optimized: Scans only the 3 columns needed.
SELECT
  ss_sold_date_sk,
  ss_item_sk,
  ss_customer_sk
FROM
  `{LAB01_DATASET_ID}.store_sales`
WHERE
  ss_ticket_number = 46153516
LIMIT 10;
"""

# Run the queries
print("--- Running Un-Optimized Query (SELECT *) ---")
a1_job1 = run_bq_query(a1_unopt_query)

print("\n--- Running Optimized Query (SELECT specific_columns) ---")
a1_job2 = run_bq_query(a1_opt_query)

# Show both jobs side by side
if a1_job1 and a1_job2:
    compare_bq_jobs(a1_job1[0], a1_job2[0])

### 2.  Filtering Early with Conditional Aggregation

**The Anti-Pattern:** A common but highly inefficient pattern is to join a massive table and then use a `CASE` statement inside an aggregate function (like `SUM`) to compute a value for a specific subset of data. This forces BigQuery to process the entire massive join, evaluating the condition for every single row, which is incredibly wasteful.

**The Fix:** Pre-filter and pre-aggregate the subset of data in a CTE first. This creates a very small, intermediate table. You can then `LEFT JOIN` this small table to get the result, which is orders of magnitude more efficient than processing the entire large table.

**The Scenario:** For every customer, calculate their total lifetime spending and their total spending from just January 2001.

In [ ]:

a2_unopt_query = f"""
-- Un-Optimized: Joins the entire store_sales table to customer, then
-- evaluates a CASE statement across billions of rows to find Jan 2001 sales.
SELECT
  c.c_customer_id,
  SUM(s.ss_net_paid) AS total_lifetime_spend,
  SUM(CASE
      WHEN s.ss_sold_date_sk BETWEEN 2451911 AND 2451941 THEN s.ss_net_paid
      ELSE 0
    END) AS total_jan_2001_spend
FROM
  `{LAB01_DATASET_ID}.customer` AS c
LEFT JOIN
  `{LAB01_DATASET_ID}.store_sales` AS s
  ON c.c_customer_sk = s.ss_customer_sk
GROUP BY
  c.c_customer_id
LIMIT 10000;
"""

a2_opt_query = f"""
-- Optimized: Pre-aggregates the small slice of Jan 2001 sales first,
-- then joins the small result back to the customer table.
WITH JanSales AS (
  SELECT
    ss_customer_sk,
    SUM(ss_net_paid) AS total_jan_spend
  FROM
    `{LAB01_DATASET_ID}.store_sales`
  WHERE
    ss_sold_date_sk BETWEEN 2451911 AND 2451941 -- Filter is applied EARLY
  GROUP BY
    ss_customer_sk
),
TotalSales AS (
  -- Also pre-aggregating total sales for a more efficient final join
  SELECT
    ss_customer_sk,
    SUM(ss_net_paid) as total_lifetime_spend
  FROM
    `{LAB01_DATASET_ID}.store_sales`
  GROUP BY
    ss_customer_sk
)
SELECT
  c.c_customer_id,
  ts.total_lifetime_spend,
  js.total_jan_spend
FROM
  `{LAB01_DATASET_ID}.customer` AS c
LEFT JOIN
  JanSales AS js
  ON c.c_customer_sk = js.ss_customer_sk
LEFT JOIN
  TotalSales AS ts
  ON c.c_customer_sk = ts.ss_customer_sk
LIMIT 10000;
"""

# Run the queries
print("--- Running Un-Optimized Query (Filtering Late) ---")
a2_job1 = run_bq_query(a2_unopt_query)

print("\n--- Running Optimized Query (Filtering Early) ---")
a2_job2 = run_bq_query(a2_opt_query)

# Show both jobs side by side
if a2_job1 and a2_job2:
    compare_bq_jobs(a2_job1[0], a2_job2[0])


### 2.1. Trusting the Optimizer: Join Reordering

**The Scenario:** A common pattern is to join a large fact table to smaller dimension tables to filter on an attribute (e.g., find all sales for customers in a specific state). An intuitive "fix" might be to pre-join and filter the small tables in a CTE before joining them to the large table.

**The Lesson:** In this exercise, we will see if our manual optimization can beat BigQuery's built-in, cost-based query optimizer. The optimizer analyzes all tables in a query and automatically reorders the joins to be as efficient as possible.

**Expected Outcome:** The optimizer is incredibly smart. It knows to join the small `customer` and `customer_address` tables first before touching the massive `store_sales` table, even in the "un-optimized" query. You will likely see very similar performance for both queries, proving that for join ordering, you can often trust the optimizer to find the best path.

In [ ]:

# --- 2. Filtering Late vs. Filtering Early (Corrected Example) ---
# Scenario: Find the total sales for all customers in Georgia ('GA').

a2_unopt_query = f"""
-- Un-Optimized: Joins three tables, including the massive 28.8 billion row
-- store_sales table, before filtering for customers in Georgia.
SELECT
  SUM(s.ss_net_paid) as total_ga_sales
FROM
  `{LAB01_DATASET_ID}.store_sales` AS s
JOIN
  `{LAB01_DATASET_ID}.customer` AS c
  ON s.ss_customer_sk = c.c_customer_sk
JOIN
  `{LAB01_DATASET_ID}.customer_address` AS ca
  ON c.c_current_addr_sk = ca.ca_address_sk -- The necessary join to get the state
WHERE
  ca.ca_state = 'GA'; -- Filter applied after the massive joins
"""

a2_opt_query = f"""
-- Optimized: Finds the small list of Georgia customers FIRST by joining
-- the two small dimension tables, then joins that tiny list to the sales table.
WITH georgia_customers AS (
  SELECT c.c_customer_sk
  FROM
    `{LAB01_DATASET_ID}.customer` AS c
  JOIN
    `{LAB01_DATASET_ID}.customer_address` AS ca
    ON c.c_current_addr_sk = ca.ca_address_sk -- Join small tables first
  WHERE
    ca.ca_state = 'GA'
)
SELECT
  SUM(s.ss_net_paid) as total_ga_sales
FROM
  `{LAB01_DATASET_ID}.store_sales` AS s
JOIN
  georgia_customers AS gc
  ON s.ss_customer_sk = gc.c_customer_sk;
"""

# Run the queries
print("--- Running Un-Optimized Query (Filtering Late) ---")
a2_job1 = run_bq_query(a2_unopt_query)

print("\n--- Running Optimized Query (Filtering Early) ---")
a2_job2 = run_bq_query(a2_opt_query)

# Show both jobs side by side
if a2_job1 and a2_job2:
    compare_bq_jobs(a2_job1[0], a2_job2[0])

### 3. Inefficient `ORDER BY`

**The Anti-Pattern:** Running an `ORDER BY` on a massive table without a `LIMIT` clause is extremely resource-intensive. To produce a total ordering of the data, BigQuery must gather all the rows onto a single worker for a final sort. This operation is very slow and will often fail with a "resources exceeded" error.

**The Fix:** Always pair `ORDER BY` with a `LIMIT` when working with large datasets. This allows BigQuery to perform a much more efficient, distributed "top-N" sort, where only the top results from each worker need to be gathered and sorted.

**Expected Outcome:** The optimized query will complete much faster, showing a huge reduction in **Query Time** and **Slot Milliseconds**. The **Bytes Scanned** will likely be the same, as BigQuery still needs to read the full column to determine which values are the top ones.

In [ ]:
# --- 3. Inefficient ORDER BY ---

# The un-optimized query below is designed to fail or take a very long time.
# It demonstrates an operation that should not be run on a large dataset.
# You may want to cancel it after observing its high slot usage.

a3_unopt_query = f"""
-- Un-Optimized: Attempts to sort the entire 28.8 billion row table.
-- This will consume massive resources and likely fail.
SELECT
  ss_net_paid
FROM
  `{LAB01_DATASET_ID}.store_sales`
ORDER BY
  ss_net_paid DESC;
"""

a3_opt_query = f"""
-- Optimized: Efficiently finds the top 100 sales using a LIMIT.
-- This allows BigQuery to perform a distributed top-N sort.
SELECT
  ss_net_paid
FROM
  `{LAB01_DATASET_ID}.store_sales`
ORDER BY
  ss_net_paid DESC
LIMIT 100;
"""

# Run the queries
print("--- Running Un-Optimized Query (ORDER BY without LIMIT) ---")
a3_job1 = run_bq_query(a3_unopt_query)

print("\n--- Running Optimized Query (ORDER BY with LIMIT) ---")
a3_job2 = run_bq_query(a3_opt_query)

# Show both jobs side by side
if a3_job1 and a3_job2:
    compare_bq_jobs(a3_job1[0], a3_job2[0])

### 4.  Accidental Many-to-Many Join

**The Anti-Pattern:** This is a subtle but dangerous mistake. When you join multiple large "fact" tables on a common, non-unique key (like a date), you create a hidden Cartesian product. For each date, every web sale is joined with every store sale, leading to massively inflated, incorrect results and terrible performance.

**The Fix:** The correct pattern is to **aggregate before you join**. First, calculate the daily totals for each fact table in separate CTEs. Then, join the much smaller, pre-aggregated results. This ensures the final join is a simple and efficient one-to-one lookup.

**Expected Outcome:** The optimized query will be dramatically faster, showing a huge reduction in **Slot Milliseconds**. More importantly, it will produce the **correct, non-inflated results**.

In [ ]:
# --- 4. Accidental Many-to-Many Join ---
# The business goal is to compare sales for a single store and web site for a single day: June 2nd, 2001

a4_unopt_query = f"""
-- Un-Optimized: Creates a many-to-many join between a single store's sales
-- and a single web site's sales for one day.
SELECT
  d.d_date,
  SUM(ws.ws_net_paid_inc_tax) AS inflated_web_sales,
  SUM(ss.ss_net_paid_inc_tax) AS inflated_store_sales
FROM
  `{LAB01_DATASET_ID}.date_dim` d
JOIN
  `{LAB01_DATASET_ID}.web_sales` ws
  ON d.d_date_sk = ws.ws_sold_date_sk
JOIN
  `{LAB01_DATASET_ID}.store_sales` ss
  ON d.d_date_sk = ss.ss_sold_date_sk
WHERE
  d.d_date = '2001-06-02'
  AND ss.ss_store_sk = 25 -- Filter to a single store
  AND ws.ws_web_site_sk = 5 -- Filter to a single web site
GROUP BY
  d.d_date;
"""

a4_opt_query = f"""
-- Optimized: Aggregate the filtered data from each table *before* joining.
WITH WebSales AS (
  SELECT
    ws_sold_date_sk,
    SUM(ws_net_paid_inc_tax) AS web_sales_total
  FROM
    `{LAB01_DATASET_ID}.web_sales`
  WHERE
    ws_sold_date_sk = 2452063 -- Corresponds to 2001-06-02
    AND ws_web_site_sk = 5 -- **CORRECTION: Was 'ws_ws_web_site_sk'**
  GROUP BY
    ws_sold_date_sk
),
StoreSales AS (
  SELECT
    ss_sold_date_sk,
    SUM(ss_net_paid_inc_tax) AS store_sales_total
  FROM
    `{LAB01_DATASET_ID}.store_sales`
  WHERE
    ss_sold_date_sk = 2452063 -- Corresponds to 2001-06-02
    AND ss_store_sk = 25 -- Filter to a single store
  GROUP BY
    ss_sold_date_sk
)
SELECT
  d.d_date,
  ws.web_sales_total,
  ss.store_sales_total
FROM
  `{LAB01_DATASET_ID}.date_dim` d
LEFT JOIN
  WebSales ws ON d.d_date_sk = ws.ws_sold_date_sk
LEFT JOIN
  StoreSales ss ON d.d_date_sk = ss.ss_sold_date_sk
WHERE
  d.d_date = '2001-06-02';
"""

# Run the queries
print("--- Running Un-Optimized Query (Scoped to Store and Web Site) ---")
a4_job1 = run_bq_query(a4_unopt_query, show_query_results=True)

print("\n--- Running Optimized Query (Scoped and Pre-Aggregated) ---")
a4_job2 = run_bq_query(a4_opt_query, show_query_results=True)

# Show both jobs side by side
if a4_job1 and a4_job2:
    compare_bq_jobs(a4_job1[0], a4_job2[0])

### 5.  JavaScript UDF vs. Native SQL Function

**The Anti-Pattern:** While User-Defined Functions (UDFs) are powerful, using them for simple tasks that a native function can handle is inefficient. There is significant overhead in starting the JavaScript engine and passing data back and forth between the SQL and JS environments for every single row.

**The Fix:** Whenever a built-in SQL function exists for your task, use it. Native functions are written in C++ and are tightly integrated into the BigQuery engine, making them orders of magnitude faster.

**Expected Outcome:** You will see a significant reduction in **Slot Milliseconds** and **Query Time**, demonstrating the raw performance advantage of native functions over UDFs for common tasks.

In [ ]:
# --- 5. JavaScript UDF vs. Native SQL Function ---
# Scenario: We need to format a date into a readable string like "Monday, January 15, 2001".
# We will process all dates from the date_dim table for the year 2001.

a5_unopt_query = f"""
-- Un-Optimized: Using a JavaScript UDF for a simple date formatting task.
CREATE TEMP FUNCTION formatDate(d DATE)
RETURNS STRING
LANGUAGE js AS r\"\"\"
  const options = {{ weekday: 'long', year: 'numeric', month: 'long', day: 'numeric' }};
  return d.toLocaleDateString('en-US', options);
\"\"\";

SELECT
  d_date_sk,
  formatDate(d_date) AS formatted_date
FROM
  `{LAB01_DATASET_ID}.date_dim`
WHERE
  d_year = 2001;
"""

a5_opt_query = f"""
-- Optimized: Using the highly efficient, built-in FORMAT_DATE function.
SELECT
  d_date_sk,
  FORMAT_DATE('%A, %B %d, %Y', d_date) AS formatted_date
FROM
  `{LAB01_DATASET_ID}.date_dim`
WHERE
  d_year = 2001;
"""

# Run the queries
print("--- Running Un-Optimized Query (JavaScript UDF) ---")
a5_job1 = run_bq_query(a5_unopt_query)

print("\n--- Running Optimized Query (Native SQL Function) ---")
a5_job2 = run_bq_query(a5_opt_query)

# Show both jobs side by side
if a5_job1 and a5_job2:
    compare_bq_jobs(a5_job1[0], a5_job2[0])

### 6.  Data Exploration: `LIMIT` vs. `TABLESAMPLE`

**The Anti-Pattern:** Using `LIMIT` to get a quick preview of a large table is a common habit, but it can be inefficient. BigQuery may still need to perform a large scan to find and return the first 1,0000 rows, leading to higher-than-expected costs and delays.

**The Fix:** For a statistically representative preview of your data, use `TABLESAMPLE SYSTEM`. This function is specifically designed for cheap and fast exploration, as it only reads a small, random percentage of the underlying data blocks.

**Expected Outcome:** The `TABLESAMPLE` query will be dramatically faster and cheaper. You should see a massive reduction in both **Bytes Scanned** and **Query Time**.

In [ ]:
# --- 6. LIMIT vs. TABLESAMPLE ---

a6_unopt_query = f"""
-- Un-Optimized: Using LIMIT for exploration can still scan a lot of data.
SELECT
  *
FROM
  `{LAB01_DATASET_ID}.store_sales`
  WHERE RAND() < 0.0000005 -- Filter for a small random sample
ORDER BY
  RAND()
LIMIT 10000;
"""

a6_opt_query = f"""
-- Optimized: Using TABLESAMPLE is cheaper and faster for a random preview.
-- For a 10TB table, even a tiny percentage is enough for a sample.
SELECT
  *
FROM
  `{LAB01_DATASET_ID}.store_sales` TABLESAMPLE SYSTEM (0.00001 PERCENT);
"""

# Run the queries
print("--- Running Un-Optimized Query (LIMIT) ---")
a6_job1 = run_bq_query(a6_unopt_query)

print("\n--- Running Optimized Query (TABLESAMPLE) ---")
a6_job2 = run_bq_query(a6_opt_query)

# Show both jobs side by side
if a6_job1 and a6_job2:
    compare_bq_jobs(a6_job1[0], a6_job2[0])

### 7.  Exact vs. Approximate Aggregations

**The Anti-Pattern:** Calculating an exact `COUNT(DISTINCT)` on a column with millions or billions of unique values (a "high cardinality" column) is computationally expensive. It requires significant resources to track every unique value encountered.

**The Fix:** For use cases where a highly accurate estimate is sufficient (like dashboards or general analysis), use `APPROX_COUNT_DISTINCT`. This function uses the efficient HyperLogLog++ algorithm to provide an estimate with a very small margin of error, but with much lower computational cost.

**Expected Outcome:** You will see a significant reduction in **Slot Milliseconds**, highlighting the performance benefits of choosing approximation when perfect precision isn't a strict requirement.

In [ ]:

# --- 7. COUNT(DISTINCT) vs. APPROX_COUNT_DISTINCT ---
# We are filtering to a single year (2001) to ensure the query
# completes in a reasonable time for the lab.

a7_unopt_query = f"""
-- Un-Optimized: Exact distinct count on a high-cardinality column.
SELECT
  ss_store_sk,
  COUNT(DISTINCT ss_ticket_number) AS exact_ticket_count
FROM
  `{LAB01_DATASET_ID}.store_sales`
WHERE
  ss_sold_date_sk BETWEEN 2451911 AND 2452275 -- Filter for year 2001
GROUP BY
  ss_store_sk
ORDER BY
  ss_store_sk;
"""

a7_opt_query = f"""
-- Optimized: Approximate count is much more efficient.
SELECT
  ss_store_sk,
  APPROX_COUNT_DISTINCT(ss_ticket_number) AS approximate_ticket_count
FROM
  `{LAB01_DATASET_ID}.store_sales`
WHERE
  ss_sold_date_sk BETWEEN 2451911 AND 2452275 -- Filter for year 2001
GROUP BY
  ss_store_sk
ORDER BY
  ss_store_sk;
"""

# Run the queries
print("--- Running Un-Optimized Query (COUNT DISTINCT for 1 Year) ---")
a7_job1 = run_bq_query(a7_unopt_query)

print("\n--- Running Optimized Query (APPROX_COUNT_DISTINCT for 1 Year) ---")
a7_job2 = run_bq_query(a7_opt_query)

# Show both jobs side by side
if a7_job1 and a7_job2:
    compare_bq_jobs(a7_job1[0], a7_job2[0])

# BigQuery Optimization Challenge Labs

## Challenge 1 (15 mins)

**✉️ Inbox (1): New email!**

Subject: **URGENT!** Performance Degradation on Critical Sales Analysis Query


>Dear Google Customer Engineer,
>
> <br>
>
>We are experiencing a severe performance issue with a critical SQL query used for our business review sales dashboard that our whole C level uses on a daily basis.
>
>Our C level **cannot afford to wait more than 30 secs** to load this SQL in the dashboard which shows data up to the last minute of sales.
>
>This is a new query and we thought that BigQuery would perform much better than what we are seeing now. We might need to revisit our data warehouse solution if we do not get this fixed soon.
>
>After some internal investigation, here is what our team has noted:
>
>*"We suspect the slow performance is due to the poor performance of BQ itself, especially considering the complex UNION ALL structure, the filtering conditions (REGEXP_CONTAINS), and potentially the implicit cross-join in the second UNION ALL branch (where a join condition seems to be missing or commented out). We could try to increase slots, but we have no additional budget."*
>
>Please review the SQL below and table schemas and advise on the most effective tuning strategies (e.g., indexing, query restructuring, partitioning optimization).
>
>We look forward to your quick analysis and recommendations.
>
>
>P.S: We need to make sure we are powering our dashboard with these columns: c_last_name, c_first_name, ca_city, bought_something.ss_ticket_number, bought_something.amt, bought_something.profit
>
>Thank you,
>
><br>
>
>**John Googliani**
>
>Data Analytics Team Lead
>
>The G Company

#### Rules:

*   Do **not** increase the size of the given BQ slot reservations
* Do not run the queries on this lab using BQ on-demand
*   You can apply **any technique** you know for improving SQL queries (Just do not cache it! No cheating either! 😅)
*   You can **physically modify any table** for clustering, partitioning, etc. The **only exceptions are the large fact tables (`store_sales`, `web_sales`, `catalog_sales`)** for which we are providing a few options for you to use and reference in your SQL in order to save time.
*   Most importantly... **Have Fun!!** 😃





### BQ Setup

In [ ]:
# Drop if exists
sql = f"""DROP ASSIGNMENT `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}.{RESERVATION_NAME}-assignment`"""

try:
  run_bq_query(sql)
except:
  pass

sql = f"""DROP RESERVATION `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}`"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""CREATE RESERVATION `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}`
            OPTIONS (edition = "enterprise",
                     slot_capacity = 0,
                     autoscale_max_slots = 200);
  """

run_bq_query(sql)

In [ ]:
sql = f"""CREATE ASSIGNMENT `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}.{RESERVATION_NAME}-assignment`
            OPTIONS(assignee = "projects/{PROJECT_ID}",
                    job_type = "QUERY");
  """

run_bq_query(sql)

In [ ]:
sql = f"""ALTER PROJECT `{PROJECT_ID}`
SET OPTIONS (
 `region-US.query_runtime` = NULL
)"""

run_bq_query(sql)

In [ ]:
sql = f"""
CREATE SCHEMA IF NOT EXISTS `{PROJECT_ID}.{YOUR_WORKING_DATASET}`
"""

run_bq_query(sql)

### Case Study

#### BAD Query

In [ ]:
opt_run_history = {}

In [ ]:
sql = f"""
SELECT
  /*
  (1) SELECTING MORE COLUMNS THAN THE NECESSARY RESULTING IN SLOWER QUERY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number,
  bought_something.amt,
  bought_something.profit
  */
  * --ADDED (1)
FROM
  (
    SELECT
      ss_ticket_number,
      ss_customer_sk,
      d_dow, -- ADDED (4)
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales` store_sales,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.date_dim` date_dim,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store` store,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.household_demographics` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      /*
      (2) REMOVED A JOIN CLAUSE RESULTING IN CROSS JOIN
      */
      AND store_sales.ss_store_sk = store.s_store_sk
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      /*
      (4) REPLACED THE FILTER TO ALLOW MORE UNECESSARY DATA RESULTING IN MORE DATA TO BE AGGREGATE AND FILTERED OUT LATER ON IN THE QUERY
      AND date_dim.d_dow = 1
      */
      AND date_dim.d_dow = 1
      /*
      (3) REPLACEING A WHERE FOR A STRING USING REGEXP_CONTAINS INSTEAD RESULTING IN A MUICH LESS INEFFICIENT QUERY
      AND date_dim.d_year IN (2000, 2001, 2002)
      */
      AND REGEXP_CONTAINS(CAST(date_dim.d_date AS STRING), r'^(2000|2001|2002)-') -- ADDED (3)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk,
      d_dow -- ADDED (4)
UNION ALL -- ADDED (4)
SELECT -- ADDED (4)
      ss_ticket_number,
      ss_customer_sk,
      d_dow, -- ADDED (4)
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales` store_sales,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.date_dim` date_dim,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store` store,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.household_demographics` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      /*
      (2) REMOVED A JOIN CLAUSE RESULTING IN CROSS JOIN
      AND store_sales.ss_store_sk = store.s_store_sk
      */
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      /*
      (4) ADDED TO THE FILTER TO ALLOW MORE UNECESSARY DATA + USING UNECESSARY UNION ALL RESULTING IN MORE DATA TO BE AGGREGATE AND FILTERED OUT LATER ON IN THE QUERY
      AND date_dim.d_dow = 1
      */
      AND date_dim.d_dow = 2 -- ADDED (4)
      /*
      (3) REPLACING A WHERE FOR A STRING USING REGEXP_CONTAINS INSTEAD RESULTING IN A MUICH LESS INEFFICIENT QUERY
      AND date_dim.d_year IN (2000, 2001, 2002)
      */
      AND REGEXP_CONTAINS(CAST(date_dim.d_date AS STRING), r'^(2000|2001|2002)-') -- ADDED (3)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk,
      d_dow -- ADDED (4)
    /*
    (5) ADDING A USELESS ORDER BY RESULTING IN A SLOWER QUERY THAT WILL CHANGE THE ORDER IN THE OUTER QUERY
    */
    ORDER BY 1 DESC, 2 DESC -- ADDED (5)
  ) AS bought_something,
  `{PROJECT_ID}.{LINKED_DATASET_ID}.customer` customer,
  `{PROJECT_ID}.{LINKED_DATASET_ID}.customer_address` customer_address
WHERE
  bought_something.ss_customer_sk = customer.c_customer_sk
  AND bought_something.d_dow = 1 -- ADDED (4)
  AND customer.c_current_addr_sk = customer_address.ca_address_sk
  AND (
    substr(ca_zip, 1, 5) IN (
      '85562',
      '86375',
      '87063',
      '85888',
      '82981',
      '82173',
      '81980',
      '84000',
      '85966',
      '85994'
    )
    OR ca_state IN (
      'CA',
      'WA',
      'GA'
    )
    OR bought_something.profit > 500
  )
ORDER BY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number
"""

# Drop if exists
try:
  del opt_run_history['bad_query']
except:
  pass

job_badsql, opt_run_history['bad_query'] = run_bq_query(sql)

### Your Work Here

Query ReWrite

In [ ]:

sql = f"""
SELECT
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number,
  bought_something.amt,
  bought_something.profit
FROM
  (
    SELECT
      ss_ticket_number,
      ss_customer_sk,
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales` store_sales,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.date_dim` date_dim,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store` store,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.household_demographics` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      AND store_sales.ss_store_sk = store.s_store_sk
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      AND date_dim.d_dow = 1
      AND 1 = 1
      AND date_dim.d_year IN (2000, 2001, 2002)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk
  ) AS bought_something,
  `{PROJECT_ID}.{LINKED_DATASET_ID}.customer` customer,
  `{PROJECT_ID}.{LINKED_DATASET_ID}.customer_address` customer_address
WHERE
  bought_something.ss_customer_sk = customer.c_customer_sk
  AND customer.c_current_addr_sk = customer_address.ca_address_sk
  AND (
    substr(ca_zip, 1, 5) IN (
      '85562',
      '86375',
      '87063',
      '85888',
      '82981',
      '82173',
      '81980',
      '84000',
      '85966',
      '85994'
    )
    OR ca_state IN (
      'CA',
      'WA',
      'GA'
    )
    OR bought_something.profit > 500
  )
ORDER BY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number
"""

# Drop if exists
try:
  del opt_run_history['sql_rewrite']
except:
  pass

job_step1sql, opt_run_history['sql_rewrite'] = run_bq_query(sql)

In [ ]:
compare_bq_jobs(job_badsql, job_step1sql)

Add Partitioning + Clustering to store_sales table

In [ ]:
# Drop if exists
sql = f"""DROP TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_sales_v4`"""

try:
  run_bq_query(sql)
except:
  pass

sql = f"""CREATE TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_sales_v4`
(
  ss_sold_date_sk INT64,
  ss_sold_time_sk INT64,
  ss_item_sk INT64,
  ss_customer_sk INT64,
  ss_cdemo_sk INT64,
  ss_hdemo_sk INT64,
  ss_addr_sk INT64,
  ss_store_sk INT64,
  ss_promo_sk INT64,
  ss_ticket_number INT64,
  ss_quantity INT64,
  ss_wholesale_cost FLOAT64,
  ss_list_price FLOAT64,
  ss_sales_price FLOAT64,
  ss_ext_discount_amt FLOAT64,
  ss_ext_sales_price FLOAT64,
  ss_ext_wholesale_cost FLOAT64,
  ss_ext_list_price FLOAT64,
  ss_ext_tax FLOAT64,
  ss_coupon_amt FLOAT64,
  ss_net_paid FLOAT64,
  ss_net_paid_inc_tax FLOAT64,
  ss_net_profit FLOAT64
)
PARTITION BY
  RANGE_BUCKET(ss_sold_time_sk, GENERATE_ARRAY(28800, 75599, 30))
OPTIONS(
  description = 'Partitioned on ss_sold_time_sk column'
)"""

run_bq_query(sql)


In [ ]:
# Not executing the INSERT, we will use a pre-created table instead.
sql = f"""insert into `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_sales_v4`
select * from `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales`"""




In [ ]:

sql = f"""
SELECT
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number,
  bought_something.amt,
  bought_something.profit
FROM
  (
    SELECT
      ss_ticket_number,
      ss_customer_sk,
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales_v4` store_sales,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.date_dim` date_dim,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store` store,
      `{PROJECT_ID}.{LINKED_DATASET_ID}.household_demographics` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      AND store_sales.ss_store_sk = store.s_store_sk
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      AND date_dim.d_dow = 1
      AND 1 = 1
      AND date_dim.d_year IN (2000, 2001, 2002)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk
  ) AS bought_something,
  `{PROJECT_ID}.{LINKED_DATASET_ID}.customer` customer,
  `{PROJECT_ID}.{LINKED_DATASET_ID}.customer_address` customer_address
WHERE
  bought_something.ss_customer_sk = customer.c_customer_sk
  AND customer.c_current_addr_sk = customer_address.ca_address_sk
  AND (
    substr(ca_zip, 1, 5) IN (
      '85562',
      '86375',
      '87063',
      '85888',
      '82981',
      '82173',
      '81980',
      '84000',
      '85966',
      '85994'
    )
    OR ca_state IN (
      'CA',
      'WA',
      'GA'
    )
    OR bought_something.profit > 500
  )
ORDER BY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number
"""

# Drop if exists
try:
  del opt_run_history['fact_clus_and_part']
except:
  pass

job_step2sql, opt_run_history['fact_clus_and_part'] = run_bq_query(sql)

In [ ]:
compare_bq_jobs(job_badsql, job_step2sql)

Clustering the tables customer and customer_address

In [ ]:
# Drop if exists
sql = f"""DROP TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1`"""

try:
  run_bq_query(sql)
except:
  pass


sql = f"""CREATE TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1`
(
  ca_address_sk INT64,
  ca_address_id STRING,
  ca_street_number STRING,
  ca_street_name STRING,
  ca_street_type STRING,
  ca_suite_number STRING,
  ca_city STRING,
  ca_county STRING,
  ca_state STRING,
  ca_zip STRING,
  ca_country STRING,
  ca_gmt_offset FLOAT64,
  ca_location_type STRING
)
CLUSTER BY
  ca_address_sk
OPTIONS(
  description = 'Clustered on ca_address_sk column'
)"""

run_bq_query(sql)

In [ ]:
sql = f"""insert into `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1`
select * from `{PROJECT_ID}.{LINKED_DATASET_ID}.customer_address`"""

run_bq_query(sql)

In [ ]:
# Drop if exists
sql = f"""DROP TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1`"""

try:
  run_bq_query(sql)
except:
  pass


sql = f"""CREATE TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1`
(
  c_customer_sk INT64,
  c_customer_id STRING,
  c_current_cdemo_sk INT64,
  c_current_hdemo_sk INT64,
  c_current_addr_sk INT64,
  c_first_shipto_date_sk INT64,
  c_first_sales_date_sk INT64,
  c_salutation STRING,
  c_first_name STRING,
  c_last_name STRING,
  c_preferred_cust_flag STRING,
  c_birth_day INT64,
  c_birth_month INT64,
  c_birth_year INT64,
  c_birth_country STRING,
  c_login STRING,
  c_email_address STRING,
  c_last_review_date_sk INT64
)
CLUSTER BY
  c_customer_sk, c_current_addr_sk
OPTIONS(
  description = 'Clustered on c_customer_sk, c_current_addr_sk columns'
)"""

run_bq_query(sql)

In [ ]:
sql = f"""insert into `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1`
select * from `{PROJECT_ID}.{LINKED_DATASET_ID}.customer`"""

run_bq_query(sql)

In [ ]:
# Drop if exists
sql = f"""DROP TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1`"""

try:
  run_bq_query(sql)
except:
  pass


sql = f"""CREATE TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1`
(
  d_date_sk INT64,
  d_date_id STRING,
  d_date DATE,
  d_month_seq INT64,
  d_week_seq INT64,
  d_quarter_seq INT64,
  d_year INT64,
  d_dow INT64,
  d_moy INT64,
  d_dom INT64,
  d_qoy INT64,
  d_fy_year INT64,
  d_fy_quarter_seq INT64,
  d_fy_week_seq INT64,
  d_day_name STRING,
  d_quarter_name STRING,
  d_holiday STRING,
  d_weekend STRING,
  d_following_holiday STRING,
  d_first_dom INT64,
  d_last_dom INT64,
  d_same_day_ly INT64,
  d_same_day_lq INT64,
  d_current_day STRING,
  d_current_week STRING,
  d_current_month STRING,
  d_current_quarter STRING,
  d_current_year STRING,
  PRIMARY KEY (d_date_sk) NOT ENFORCED
)
CLUSTER BY d_date_sk
OPTIONS(
  description = 'Clustered on d_date_sk columns'
)"""

run_bq_query(sql)

In [ ]:
sql = f"""insert into `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1`
select * from `{PROJECT_ID}.{LINKED_DATASET_ID}.date_dim`"""

run_bq_query(sql)

In [ ]:
# Drop if exists
sql = f"""DROP TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1`"""

try:
  run_bq_query(sql)
except:
  pass


sql = f"""CREATE TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1`
(
  s_store_sk INT64,
  s_store_id STRING,
  s_rec_start_date DATE,
  s_rec_end_date DATE,
  s_closed_date_sk INT64,
  s_store_name STRING,
  s_number_employees INT64,
  s_floor_space INT64,
  s_hours STRING,
  s_manager STRING,
  s_market_id INT64,
  s_geography_class STRING,
  s_market_desc STRING,
  s_market_manager STRING,
  s_division_id INT64,
  s_division_name STRING,
  s_company_id INT64,
  s_company_name STRING,
  s_street_number STRING,
  s_street_name STRING,
  s_street_type STRING,
  s_suite_number STRING,
  s_city STRING,
  s_county STRING,
  s_state STRING,
  s_zip STRING,
  s_country STRING,
  s_gmt_offset FLOAT64,
  s_tax_percentage FLOAT64
)
CLUSTER BY s_store_sk
OPTIONS(
  description = 'Clustered on s_store_sk column'
)"""

run_bq_query(sql)

In [ ]:
sql = f"""insert into `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1`
select * from `{PROJECT_ID}.{LINKED_DATASET_ID}.store`"""

run_bq_query(sql)

In [ ]:
# Drop if exists
sql = f"""DROP TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1`"""

try:
  run_bq_query(sql)
except:
  pass


sql = f"""CREATE TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1`
(
  hd_demo_sk INT64,
  hd_income_band_sk INT64,
  hd_buy_potential STRING,
  hd_dep_count INT64,
  hd_vehicle_count INT64
)
CLUSTER BY hd_demo_sk
OPTIONS(
  description = 'Clustered on hd_demo_sk column'
)"""

run_bq_query(sql)

In [ ]:
sql = f"""insert into `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1`
select * from `{PROJECT_ID}.{LINKED_DATASET_ID}.household_demographics`"""

run_bq_query(sql)

In [ ]:
sql = f"""
SELECT
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number,
  bought_something.amt,
  bought_something.profit
FROM
  (
    SELECT
      ss_ticket_number,
      ss_customer_sk,
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales_v4` store_sales,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1` date_dim,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1` store,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      AND store_sales.ss_store_sk = store.s_store_sk
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      AND date_dim.d_dow = 1
      AND 1 = 1
      AND date_dim.d_year IN (2000, 2001, 2002)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk
  ) AS bought_something,
  `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1` customer,
  `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1` customer_address
WHERE
  bought_something.ss_customer_sk = customer.c_customer_sk
  AND customer.c_current_addr_sk = customer_address.ca_address_sk
  AND (
    substr(ca_zip, 1, 5) IN (
      '85562',
      '86375',
      '87063',
      '85888',
      '82981',
      '82173',
      '81980',
      '84000',
      '85966',
      '85994'
    )
    OR ca_state IN (
      'CA',
      'WA',
      'GA'
    )
    OR bought_something.profit > 500
  )
ORDER BY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number
"""

# Drop if exists
try:
  del opt_run_history['dimension_clu_plus_part']
except:
  pass

job_step3sql, opt_run_history['dimension_clu_plus_part'] = run_bq_query(sql)

In [ ]:
compare_bq_jobs(job_badsql, job_step3sql)

Add PKs and FKs

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_sales_v4`
ADD PRIMARY KEY (ss_item_sk, ss_ticket_number) NOT ENFORCED"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1`
ADD PRIMARY KEY (c_customer_sk) NOT ENFORCED"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1`
ADD PRIMARY KEY (ca_address_sk) NOT ENFORCED"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1`
ADD PRIMARY KEY (s_store_sk) NOT ENFORCED"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""ALTER TABLE `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1`
ADD PRIMARY KEY (hd_demo_sk) NOT ENFORCED"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""ALTER table `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales_v4`
ADD FOREIGN KEY(ss_sold_date_sk) references `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1`(d_date_sk)
 NOT ENFORCED,
ADD FOREIGN KEY(ss_store_sk) references `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1`(s_store_sk)
 NOT ENFORCED,
ADD FOREIGN KEY(ss_hdemo_sk) references `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1`(hd_demo_sk)
 NOT ENFORCED"""

# Removing is because our LINKED dataset is READ-ONLY. So, we cannot create the PKs/FKs
# run_bq_query(sql)



In [ ]:
sql = f"""
SELECT
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number,
  bought_something.amt,
  bought_something.profit
FROM
  (
    SELECT
      ss_ticket_number,
      ss_customer_sk,
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales_v4` store_sales,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1` date_dim,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1` store,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      AND store_sales.ss_store_sk = store.s_store_sk
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      AND date_dim.d_dow = 1
      AND 1 = 1
      AND date_dim.d_year IN (2000, 2001, 2002)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk
  ) AS bought_something,
  `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1` customer,
  `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1` customer_address
WHERE
  bought_something.ss_customer_sk = customer.c_customer_sk
  AND customer.c_current_addr_sk = customer_address.ca_address_sk
  AND (
    substr(ca_zip, 1, 5) IN (
      '85562',
      '86375',
      '87063',
      '85888',
      '82981',
      '82173',
      '81980',
      '84000',
      '85966',
      '85994'
    )
    OR ca_state IN (
      'CA',
      'WA',
      'GA'
    )
    OR bought_something.profit > 500
  )
ORDER BY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number
"""

# Drop if exists
try:
  del opt_run_history['adding_dim_pks']
except:
  pass

# Removing is because our LINKED dataset is READ-ONLY. So, we cannot create the PKs/FKs
job_step4sql, opt_run_history['adding_dim_pks'] = run_bq_query(sql)

In [ ]:
compare_bq_jobs(job_badsql, job_step4sql)

Advanced Query RunTime (no gain observed)

In [ ]:
sql = f"""ALTER PROJECT `{PROJECT_ID}`
SET OPTIONS (
 `region-US.query_runtime` = "advanced"
)"""

run_bq_query(sql)


In [ ]:
from time import sleep

sql = f"""
SELECT
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number,
  bought_something.amt,
  bought_something.profit
FROM
  (
    SELECT
      ss_ticket_number,
      ss_customer_sk,
      sum(ss_ext_sales_price) AS amt,
      sum(ss_net_profit) AS profit
    FROM
      `{PROJECT_ID}.{LINKED_DATASET_ID}.store_sales_v4` store_sales,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.date_dim_v1` date_dim,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.store_v1` store,
      `{PROJECT_ID}.{YOUR_WORKING_DATASET}.household_demographics_v1` household_demographics
    WHERE
      store_sales.ss_sold_date_sk = date_dim.d_date_sk
      AND store_sales.ss_store_sk = store.s_store_sk
      AND store_sales.ss_hdemo_sk = household_demographics.hd_demo_sk
      AND (
        household_demographics.hd_dep_count = 5
        OR household_demographics.hd_vehicle_count > 2
      )
      AND date_dim.d_dow = 1
      AND 1 = 1
      AND date_dim.d_year IN (2000, 2001, 2002)
      AND store.s_county IN (
        'Williamson County',
        'Franklin Parish',
        'Bronx County',
        'Orange County'
      )
    GROUP BY
      ss_ticket_number,
      ss_customer_sk
  ) AS bought_something,
  `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_v1` customer,
  `{PROJECT_ID}.{YOUR_WORKING_DATASET}.customer_address_v1` customer_address
WHERE
  bought_something.ss_customer_sk = customer.c_customer_sk
  AND customer.c_current_addr_sk = customer_address.ca_address_sk
  AND (
    substr(ca_zip, 1, 5) IN (
      '85562',
      '86375',
      '87063',
      '85888',
      '82981',
      '82173',
      '81980',
      '84000',
      '85966',
      '85994'
    )
    OR ca_state IN (
      'CA',
      'WA',
      'GA'
    )
    OR bought_something.profit > 500
  )
ORDER BY
  c_last_name,
  c_first_name,
  ca_city,
  bought_something.ss_ticket_number
"""

sleep(120)

# Drop if exists
try:
  del opt_run_history['advanced_runtime']
except:
  pass

job_step5sql, opt_run_history['advanced_runtime'] = run_bq_query(sql)

In [ ]:
compare_bq_jobs(job_badsql, job_step5sql)

In [ ]:
import pandas as pd

df = pd.DataFrame(opt_run_history)
df = df.T
df = df.reset_index()
df

In [ ]:
import plotly.express as px

fig = px.bar(df, x='index', y='duration', title="Results Comparison",  labels={'index':'Performance Technique', 'duration': 'Duration (sec)'})
fig.show()

### Challenge 1 Cleanup

Make sure to uncomment the code below.

In [ ]:
# Drop if exists
sql = f"""DROP ASSIGNMENT `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}.{RESERVATION_NAME}-assignment`"""

try:
  run_bq_query(sql)
except:
  pass

sql = f"""DROP RESERVATION `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}`"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
sql = f"""ALTER PROJECT `{PROJECT_ID}`
SET OPTIONS (
 `region-US.query_runtime` = NULL
)"""

run_bq_query(sql)

## Challenge 2 (30 mins)

---

### **✉️ Inbox (1) - New email!:**

**Subject:** *Re: URGENT: Performance Degradation on Critical Sales Analysis Query // **New Performance Issue: Multi-stage Customer Analysis Query***


> *Dear Google Customer Engineer,*
>
> <br>
>
> *Thank you so much for your help with our sales dashboard query. Your recommendations were spot on! The C-suite is thrilled with the new performance, and the dashboard now loads well under our 30-second target.*
>
> <br>
>
> *Unfortunately, we've hit another performance bottleneck with a different, more complex query used for a critical customer behavior analysis report. The current execution time is over **12 minutes**, which is completely unworkable for our analysts.*
>
> <br>
>
> *This query performs a multi-stage analysis to help us understand our best customers. In short, it does the following:*
>
> 1.  *Finds our most **frequently sold items** in stores between 1999 and 2002.*
> 2.  *Identifies our **top store customers** (the 5th percentile) based on their total sales during that same period.*
> 3.  *Finally, it calculates the total **web and catalog sales** in December 2002, specifically for these top customers buying those frequent items.*
>
> <br>
>
> *Our goal is to get this query to run in **under a minute and a half**.*
>
> *Could you please review this new query and provide your optimization expertise? Your help on the last one was invaluable, and we'd appreciate your support again here.*
>
> <br>
>
> *Thank you,*
>
> ***John Googliani***
>
> *Data Analytics Team Lead*
>
> *The G Company*

### BQ Setup

In [ ]:
# Drop if exists
sql = f"""DROP ASSIGNMENT `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}.{RESERVATION_NAME}-assignment`"""

try:
  run_bq_query(sql)
except:
  pass

sql = f"""DROP RESERVATION `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}`"""

try:
  run_bq_query(sql)
except:
  pass

In [ ]:
# Create a reservation with 0 baseline and 1,000 max autoscale slots.,
sql = f"""CREATE RESERVATION `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}`
            OPTIONS (edition = "enterprise",
                     slot_capacity = 0,
                     autoscale_max_slots = 1000);
  """

run_bq_query(sql)

In [ ]:
# Assign the current project to the new reservation
sql = f"""CREATE ASSIGNMENT `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}.{RESERVATION_NAME}-assignment`
            OPTIONS(assignee = "projects/{PROJECT_ID}",
                    job_type = "QUERY");
  """

run_bq_query(sql)

In [ ]:
# Disable BQ advanced query runtime
sql = f"""ALTER PROJECT `{PROJECT_ID}`
SET OPTIONS (
 `region-{REGION}.query_runtime` = NULL
)"""

run_bq_query(sql)

In [ ]:
# Dictionary to store executions
opt_run_history_c2 = {}

### Get started

**Challenge rules:**
- Use a 1,000 max slot Enterprise reservation, no idle slot sharing
- No caching or materialization
- You can modify the query as long as the business logic and query results are the same
- You can find the tables in the dataset [linked earlier in this notebook](https://console.cloud.google.com/bigquery/analytics-hub/exchanges/projects/720965328418/locations/us/dataExchanges/argolis_shared_data_1840b70ffcb/listings/bigquery_optimization_lab_199a6715d4a).
- You can create your own versions of the tables (e.g. with different partitioning or clustering specs) as needed. The fact tables (`catalog_sales`,`store_sales`, `web_sales`) are very large and it will take a long time to recreate. To speed things up, you can use any of the already partitioned/clustered tables with the suffix `_v...`.

<br>

---

<br>

####Original query:

```sql
WITH
  frequent_ss_items_1999 AS (
    # Find frequently sold items in 1999
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `<your_project_id>.<your_dataset_id>.store_sales`,
      `<your_project_id>.<your_dataset_id>.date_dim`,
      `<your_project_id>.<your_dataset_id>.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year IN (1999)
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items_2000 AS (
    # Find frequently sold items in 2000
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `<your_project_id>.<your_dataset_id>.store_sales`,
      `<your_project_id>.<your_dataset_id>.date_dim`,
      `<your_project_id>.<your_dataset_id>.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year = 2000
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items_2001 AS (
    # Find frequently sold items in 2001
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `<your_project_id>.<your_dataset_id>.store_sales`,
      `<your_project_id>.<your_dataset_id>.date_dim`,
      `<your_project_id>.<your_dataset_id>.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year = 2001
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items_2002 AS (
    # Find frequently sold items in 2002
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `<your_project_id>.<your_dataset_id>.store_sales`,
      `<your_project_id>.<your_dataset_id>.date_dim`,
      `<your_project_id>.<your_dataset_id>.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year = 2002
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items AS (
    # Combine the frequently sold items in the years above
    SELECT * FROM frequent_ss_items_1999
    UNION ALL
    SELECT * FROM frequent_ss_items_2000
    UNION ALL
    SELECT * FROM frequent_ss_items_2001
    UNION ALL
    SELECT * FROM frequent_ss_items_2002
  ),
  max_store_sales AS (
    # Calculates the max store sales per customer in the same time period
    SELECT
      MAX(csales) tpcds_cmax
    FROM
      (
        SELECT
          c_customer_sk,
          SUM(ss_quantity * ss_sales_price) csales
        FROM
          `<your_project_id>.<your_dataset_id>.store_sales`
        CROSS JOIN
          `<your_project_id>.<your_dataset_id>.date_dim`
        CROSS JOIN
          `<your_project_id>.<your_dataset_id>.customer`
        WHERE
          ss_customer_sk = c_customer_sk
          AND ss_sold_date_sk = d_date_sk
          AND d_year IN (
            1999,
            1999 + 1,
            1999 + 2,
            1999 + 3)
        GROUP BY
          c_customer_sk
      )
  ),
  best_ss_customer AS (
    # Finds the best store customers in the top 5% of sales
    SELECT
      c_customer_sk,
      SUM(ss_quantity * ss_sales_price) ssales
    FROM
      `<your_project_id>.<your_dataset_id>.store_sales`,
      `<your_project_id>.<your_dataset_id>.customer`
    WHERE
      ss_customer_sk = c_customer_sk
    GROUP BY
      c_customer_sk
    HAVING
      SUM(ss_quantity * ss_sales_price) > (95 / 100.0) * (SELECT * FROM max_store_sales)
  )

# Calculate web and catalog sales from December 2002
# made by the best store customers buying the most frequent store items.
SELECT
  SUM(sales)
FROM
  (
    SELECT
      cs_quantity * cs_list_price sales
    FROM
      `<your_project_id>.<your_dataset_id>.catalog_sales`,
      `<your_project_id>.<your_dataset_id>.date_dim`
    WHERE
      d_year = 2002
      AND d_moy = 12
      AND cs_sold_date_sk = d_date_sk
      AND cs_item_sk IN (
        SELECT
          item_sk
        FROM
          frequent_ss_items
      )
      AND cs_bill_customer_sk IN (
        SELECT
          c_customer_sk
        FROM
          best_ss_customer
      )
    UNION ALL
    SELECT
      ws_quantity * ws_list_price sales
    FROM
      `<your_project_id>.<your_dataset_id>.web_sales`,
      `<your_project_id>.<your_dataset_id>.date_dim`
    WHERE
      d_year = 2002
      AND d_moy = 12
      AND ws_sold_date_sk = d_date_sk
      AND ws_item_sk IN (
        SELECT
          item_sk
        FROM
          frequent_ss_items
      )
      AND ws_bill_customer_sk IN (
        SELECT
          c_customer_sk
        FROM
          best_ss_customer
      )
  )
LIMIT
  100;
```
---

**Note: this initial query runs on non-partitioned, non-clustered data and it will take about 12 minues. You don't need to run it during the lab**

In [ ]:
# [Query 0] Initial attempt with unpartitioned dataset
# DO NOT RUN DURING THE LAB - it will take more than 10 min

C_DATASET_ID = f"{PROJECT_ID}.{LINKED_DATASET_ID}"

q0_query = f"""WITH
  frequent_ss_items_1999 AS (
    # Find frequently sold items in 1999
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `{C_DATASET_ID}.store_sales`,
      `{C_DATASET_ID}.date_dim`,
      `{C_DATASET_ID}.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year IN (1999)
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items_2000 AS (
    # Find frequently sold items in 2000
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `{C_DATASET_ID}.store_sales`,
      `{C_DATASET_ID}.date_dim`,
      `{C_DATASET_ID}.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year = 2000
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items_2001 AS (
    # Find frequently sold items in 2001
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `{C_DATASET_ID}.store_sales`,
      `{C_DATASET_ID}.date_dim`,
      `{C_DATASET_ID}.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year = 2001
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items_2002 AS (
    # Find frequently sold items in 2002
    SELECT
      i_item_desc,
      i_item_sk item_sk,
      d_date solddate,
      COUNT(*) cnt
    FROM
      `{C_DATASET_ID}.store_sales`,
      `{C_DATASET_ID}.date_dim`,
      `{C_DATASET_ID}.item`
    WHERE
      ss_sold_date_sk = d_date_sk
      AND ss_item_sk = i_item_sk
      AND d_year = 2002
    GROUP BY
      i_item_desc,
      i_item_sk,
      d_date
    HAVING
      COUNT(*) > 4
  ),
  frequent_ss_items AS (
    # Combine the frequently sold items in the years above
    SELECT * FROM frequent_ss_items_1999
    UNION ALL
    SELECT * FROM frequent_ss_items_2000
    UNION ALL
    SELECT * FROM frequent_ss_items_2001
    UNION ALL
    SELECT * FROM frequent_ss_items_2002
  ),
  max_store_sales AS (
    # Calculates the max store sales per customer in the same time period
    SELECT
      MAX(csales) tpcds_cmax
    FROM
      (
        SELECT
          c_customer_sk,
          SUM(ss_quantity * ss_sales_price) csales
        FROM
          `{C_DATASET_ID}.store_sales`
        CROSS JOIN
          `{C_DATASET_ID}.date_dim`
        CROSS JOIN
          `{C_DATASET_ID}.customer`
        WHERE
          ss_customer_sk = c_customer_sk
          AND ss_sold_date_sk = d_date_sk
          AND d_year IN (
            1999,
            1999 + 1,
            1999 + 2,
            1999 + 3)
        GROUP BY
          c_customer_sk
      )
  ),
  best_ss_customer AS (
    # Finds the best store customers in the top 5% of sales
    SELECT
      c_customer_sk,
      SUM(ss_quantity * ss_sales_price) ssales
    FROM
      `{C_DATASET_ID}.store_sales`,
      `{C_DATASET_ID}.customer`
    WHERE
      ss_customer_sk = c_customer_sk
    GROUP BY
      c_customer_sk
    HAVING
      SUM(ss_quantity * ss_sales_price) > (95 / 100.0) * (SELECT * FROM max_store_sales)
  )

# Calculate web and catalog sales from December 2002
# made by the best store customers buying the most frequent store items.
SELECT
  SUM(sales)
FROM
  (
    SELECT
      cs_quantity * cs_list_price sales
    FROM
      `{C_DATASET_ID}.catalog_sales`,
      `{C_DATASET_ID}.date_dim`
    WHERE
      d_year = 2002
      AND d_moy = 12
      AND cs_sold_date_sk = d_date_sk
      AND cs_item_sk IN (
        SELECT
          item_sk
        FROM
          frequent_ss_items
      )
      AND cs_bill_customer_sk IN (
        SELECT
          c_customer_sk
        FROM
          best_ss_customer
      )
    UNION ALL
    SELECT
      ws_quantity * ws_list_price sales
    FROM
      `{C_DATASET_ID}.web_sales`,
      `{C_DATASET_ID}.date_dim`
    WHERE
      d_year = 2002
      AND d_moy = 12
      AND ws_sold_date_sk = d_date_sk
      AND ws_item_sk IN (
        SELECT
          item_sk
        FROM
          frequent_ss_items
      )
      AND ws_bill_customer_sk IN (
        SELECT
          c_customer_sk
        FROM
          best_ss_customer
      )
  )
LIMIT
  100;
"""

# DO NOT RUN - takes over 10 min
#q0, opt_run_history_c2['q0_original'] = run_bq_query(q0_query, show_query_results=True)

In [ ]:
# Instead of running the initial long query, we'll capture the execution stats in a
# dictionary for comparison

q0_stats = {
    "id": "2ffb94ef-165d-4ae8-a2cd-4acbd84cfcbf",
    "rows_returned": 1,
    "query_time": 675.62,
    "bytes_scanned": 1972736671505,
    "slot_ms": 654427904
}

Output:

```
Running query in project '__________', location 'us'...
Job started. ID: 2ffb94ef-165d-4ae8-a2cd-4acbd84cfcbf
Job 2ffb94ef-165d-4ae8-a2cd-4acbd84cfcbf finished with state: DONE
------------------------------
 Rows Returned:    1
 Duration:      675.62 s
 Bytes Scanned:    1.8 TiB (1,972,736,671,505 bytes)
 Slot Time:      654,427,904 ms
------------------------------

Query Results (first 10 rows):
------------------------------
         f0_
0  293712.02
------------------------------
```

### Your Work Here:

In [ ]:
# Your work here...

### Challenge 2 Cleanup

In [ ]:
# Drop assignment and reservation if they exist
sql = f"""DROP ASSIGNMENT `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}.{RESERVATION_NAME}-assignment`"""

try:
  run_bq_query(sql)
except:
  pass

sql = f"""DROP RESERVATION `{PROJECT_ID}.region-{REGION}.{RESERVATION_NAME}`"""

try:
  run_bq_query(sql)
except:
  pass